In [ ]:
import requests
import pandas as pd
import os
import json
from datetime import datetime, timedelta
from dotenv import load_dotenv

# 1. 환경 변수 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# 키가 없는 경우 테스트용 처리
if not API_KEY:
    print("⚠️ 경고: .env 파일에서 SEOUL_API_KEY를 찾을 수 없습니다. (테스트용 키 사용 시도)")
    # 로그에 있던 키로 잠시 테스트 (보안상 실제 파일에는 본인 키를 꼭 넣으세요)
    API_KEY = "6f6e596542617a7533366d6278685a" 

# 2. 요청 URL 구성 (파라미터 등은 동일)
SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 5
target_date = "20260201" # 로그에서 확인된 날짜로 고정 테스트

url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{target_date}"

print(f"🚀 API 재요청 시작: {url}")

try:
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    # 3. [수정됨] JSON 구조 파싱 로직 변경
    # 구조: response -> body -> items -> item
    try:
        # 데이터 위치 접근
        items = data['response']['body']['items']['item']
        
        # 데이터프레임 변환
        df = pd.DataFrame(items)
        
        print("\n" + "="*50)
        print("✅ 데이터 파싱 성공! (상위 5개 행)")
        print("="*50)
        print(df.head())
        
        print("\n" + "="*50)
        print("📊 컬럼 데이터 타입 및 매칭 확인")
        print("="*50)
        
        # 주요 컬럼 값 예시 확인
        check_cols = ['pasngDe', 'pasngHr', 'stnNm', 'rideNope', 'gffNope', 'trnscdUserSeCdNm']
        
        for col in check_cols:
            if col in df.columns:
                val = df[col].iloc[2] # 3번째 데이터(일반) 예시
                print(f" - {col}: {val} (Type: {type(val).__name__})")
            else:
                print(f" - {col}: ❌ 없음")

        # 4. 분석가 코멘트
        print("\n[👨‍💻 분석가 코멘트]")
        print("1. 승/하차 인원(rideNope, gffNope)이 숫자(int)로 잘 들어옵니다.")
        print("2. 사용자 구분(trnscdUserSeCdNm)이 '일반', '청소년' 등으로 명확합니다.")
        print("3. 이제 이 구조대로 DB에 적재하면 됩니다.")

    except KeyError as e:
        print(f"❌ 파싱 오류: JSON 구조가 예상과 다릅니다. 키를 찾을 수 없음: {e}")
        # 디버깅을 위해 키 목록 출력
        if 'response' in data:
            print("response 키 내부:", data['response'].keys())
            if 'body' in data['response']:
                print("body 키 내부:", data['response']['body'].keys())

except Exception as e:
    print(f"❌ 통신 또는 기타 오류: {e}")

🚀 API 재요청 시작: http://openapi.seoul.go.kr:8088/6f6e596542617a7533366d6278685a/json/getStnPsgr/1/5/20260201

✅ 데이터 파싱 성공! (상위 5개 행)
    pasngDe pasngHr lineNm stnCd stnNo stnNm trnscdSeCd trnscdSeCdNm  \
0  20260201      23    8호선  2828   821   남위례          2         후불카드   
1  20260201      23    8호선  2828   821   남위례          2         후불카드   
2  20260201      23    8호선  2828   821   남위례          2         후불카드   
3  20260201      23    8호선  2828   821   남위례          1         선불카드   
4  20260201      23    8호선  2828   821   남위례          1         선불카드   

  trnscdUserSeCd trnscdUserSeCdNm  rideNope  gffNope   crtrYmd  
0            100              우대권         0        0  20260201  
1             04              청소년         1        3  20260201  
2             01               일반        18       58  20260201  
3             99               기타         0        2  20260201  
4            100              우대권         0        2  20260201  

📊 컬럼 데이터 타입 및 매칭 확인
 - pasngDe: 20260201 (Type

In [4]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 1. 환경 변수 및 설정
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# (테스트용) 키가 로드되지 않았을 때를 대비해 로그에 있던 키 사용
if not API_KEY:
    API_KEY = "6f6e596542617a7533366d6278685a"

SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 5
target_date = "20260201" # 테스트용 날짜

# 2. API 호출
url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{target_date}"
response = requests.get(url)
data = response.json()

# 3. 데이터프레임 변환
try:
    items = data['response']['body']['items']['item']
    df = pd.DataFrame(items)

    # ---------------------------------------------------------
    # 🎨 [Display Styling] 분석가를 위한 깔끔한 출력 설정
    # ---------------------------------------------------------
    
    # (1) 모든 컬럼 출력 허용 (중간에 ... 생략 방지)
    pd.set_option('display.max_columns', None)
    
    # (2) 컬럼 너비 자동 조절
    pd.set_option('display.width', 1000)
    
    # (3) 숫자 가독성 (천 단위 콤마 등 필요시 설정, 현재는 원본 유지)
    # pd.options.display.float_format = '{:,.0f}'.format

    # (4) 컬럼명 한글 별칭(Alias) 매핑하여 보기 편하게 출력 (원본 DF는 유지)
    column_mapping = {
        'pasngDe': '일자',
        'pasngHr': '시간',
        'lineNm': '호선',
        'stnNm': '역명',
        'trnscdUserSeCdNm': '사용자유형',
        'rideNope': '승차',
        'gffNope': '하차'
    }
    
    print(f"📊 [데이터 미리보기] 날짜: {target_date} | 총 {len(df)}건 조회됨")
    
    # 주요 분석 컬럼만 먼저 보고 싶다면:
    # display(df[column_mapping.keys()].rename(columns=column_mapping))
    
    # 전체 원본 데이터를 보고 싶다면:
    display(df)
    
    # 데이터 타입 확인
    print("\n📋 [컬럼별 데이터 타입]")
    print(df.dtypes)

except KeyError:
    print("❌ 데이터 파싱 실패: API 응답 구조를 다시 확인해주세요.")

📊 [데이터 미리보기] 날짜: 20260201 | 총 5건 조회됨


,pasngDe,pasngHr,lineNm,stnCd,stnNo,stnNm,trnscdSeCd,trnscdSeCdNm,trnscdUserSeCd,trnscdUserSeCdNm,rideNope,gffNope,crtrYmd
0,20260201,23,8호선,2828,821,남위례,2,후불카드,100,우대권,0,0,20260201
1,20260201,23,8호선,2828,821,남위례,2,후불카드,04,청소년,1,3,20260201
2,20260201,23,8호선,2828,821,남위례,2,후불카드,01,일반,18,58,20260201
3,20260201,23,8호선,2828,821,남위례,1,선불카드,99,기타,0,2,20260201
4,20260201,23,8호선,2828,821,남위례,1,선불카드,100,우대권,0,2,20260201



📋 [컬럼별 데이터 타입]
pasngDe             object
pasngHr             object
lineNm              object
stnCd               object
stnNo               object
stnNm               object
trnscdSeCd          object
trnscdSeCdNm        object
trnscdUserSeCd      object
trnscdUserSeCdNm    object
rideNope             int64
gffNope              int64
crtrYmd             object
dtype: object


In [5]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import time

# 1. 환경 변수 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# (테스트용 키 설정)
if not API_KEY:
    API_KEY = "6f6e596542617a7533366d6278685a"

# 2. 설정: 데이터를 충분히 긁어오기 위해 반복 호출
# 보통 한 역에 여러 승객 타입이 존재하므로 3000건 정도면 대부분의 유형 포착 가능
TARGET_DATE = "20260201" 
MAX_ROWS = 3000 
BATCH_SIZE = 1000

all_items = []

print(f"🚀 사용자 유형 분석을 위해 데이터 수집 시작 (최대 {MAX_ROWS}건)...")

try:
    for start_idx in range(1, MAX_ROWS, BATCH_SIZE):
        end_idx = start_idx + BATCH_SIZE - 1
        url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/getStnPsgr/{start_idx}/{end_idx}/{TARGET_DATE}"
        
        response = requests.get(url)
        data = response.json()
        
        # 데이터 파싱
        if 'response' in data and 'body' in data['response']:
            items = data['response']['body']['items']['item']
            all_items.extend(items)
            print(f"   - {start_idx} ~ {end_idx} 구간 수집 완료")
        else:
            print("   - 데이터 끝 또는 구조 상이")
            break
            
        time.sleep(0.1) # 서버 부하 방지

    # 3. 데이터프레임 변환 및 유니크 값 추출
    df = pd.DataFrame(all_items)
    
    # 필요한 컬럼만 선택하여 중복 제거 (코드와 명칭 쌍으로 추출)
    unique_types = df[['trnscdUserSeCd', 'trnscdUserSeCdNm']].drop_duplicates().sort_values(by='trnscdUserSeCd')
    
    # 4. 결과 출력
    print("\n" + "="*40)
    print("🕵️‍♂️ 발견된 사용자 유형 (Unique User Types)")
    print("="*40)
    
    # 깔끔하게 출력
    # 인덱스 숨기고 출력 (to_string 활용)
    print(unique_types.to_string(index=False))
    
    print("\n" + "="*40)
    print(f"총 {len(unique_types)}개의 사용자 유형이 확인되었습니다.")
    print("이 코드를 기준으로 DB의 'user_type' 컬럼 길이를 설정하거나,")
    print("분석 시 필터링 조건으로 활용하세요.")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

🚀 사용자 유형 분석을 위해 데이터 수집 시작 (최대 3000건)...
   - 1 ~ 1000 구간 수집 완료
   - 1001 ~ 2000 구간 수집 완료
   - 2001 ~ 3000 구간 수집 완료

🕵️‍♂️ 발견된 사용자 유형 (Unique User Types)
trnscdUserSeCd trnscdUserSeCdNm
            01               일반
            02              어린이
            03              중고생
            04              청소년
           100              우대권
            11               일반
            12              어린이
            41            영어 일반
            42            일어 일반
            43           중국어 일반
            44           영어 어린이
            46          중국어 어린이
            99               기타

총 13개의 사용자 유형이 확인되었습니다.
이 코드를 기준으로 DB의 'user_type' 컬럼 길이를 설정하거나,
분석 시 필터링 조건으로 활용하세요.


In [7]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 1. 환경 설정
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# (테스트용) 키가 없는 경우 예비 키 사용
if not API_KEY:
    API_KEY = "6f6e596542617a7533366d6278685a"

# 2. 데이터 수집 설정 (다양한 유형 확보를 위해 1000개 요청 후 100개 추출)
SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 1000 
TARGET_DATE = "20260201" # 예시 날짜

url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{TARGET_DATE}"

print(f"🚀 데이터 수집 및 그룹핑 시작... ({url})")

try:
    response = requests.get(url)
    data = response.json()
    
    # 3. 데이터프레임 변환
    items = data['response']['body']['items']['item']
    df = pd.DataFrame(items)
    
    # 숫자형 컬럼 변환 (명세서 기준 int64)
    df['rideNope'] = df['rideNope'].astype(int)
    df['gffNope'] = df['gffNope'].astype(int)

    # ---------------------------------------------------------
    # 🛠️ [Feature Engineering] 사용자 그룹핑 로직 적용
    # ---------------------------------------------------------
    def categorize_user(code):
        # API에서 코드는 문자열(object)로 들어오므로 문자열로 비교
        if code == '01':
            return '일반'
        elif code in ['02', '03', '04']:
            return '학생/청소년'
        elif code in ['06', '100']:
            return '노인/약자'
        else:
            return '기타' # 그 외 모든 코드 (외국인, 국가유공자 등)

    # 새로운 분석용 컬럼 'UserGroup' 생성
    df['UserGroup'] = df['trnscdUserSeCd'].apply(categorize_user)
    
    # ---------------------------------------------------------
    # 🔍 결과 확인 (샘플링 100개)
    # ---------------------------------------------------------
    # 모든 컬럼 순서 유지 + 맨 앞에 UserGroup 추가하여 보기 편하게 정렬
    cols = ['UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 'stnCd', 'stnNo', 'stnNm', 
            'trnscdSeCd', 'trnscdSeCdNm', 'trnscdUserSeCd', 'trnscdUserSeCdNm', 
            'rideNope', 'gffNope', 'crtrYmd']
    
    final_df = df[cols].head(100)
    
    # 출력 설정
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    
    print(f"\n✅ 그룹핑 완료! (총 {len(final_df)}행 샘플)")
    display(final_df)
    
    # 그룹별 데이터 분포 확인 (검증용)
    print("\n📊 [검증] 샘플 데이터 그룹별 분포")
    print(final_df['UserGroup'].value_counts())

except Exception as e:
    print(f"❌ 오류 발생: {e}")

🚀 데이터 수집 및 그룹핑 시작... (http://openapi.seoul.go.kr:8088/6f6e596542617a7533366d6278685a/json/getStnPsgr/1/1000/20260201)

✅ 그룹핑 완료! (총 100행 샘플)


,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNo,stnNm,trnscdSeCd,trnscdSeCdNm,trnscdUserSeCd,trnscdUserSeCdNm,rideNope,gffNope,crtrYmd
0,노인/약자,20260201,23,8호선,2828,821,남위례,2,후불카드,100,우대권,0,0,20260201
1,학생/청소년,20260201,23,8호선,2828,821,남위례,2,후불카드,04,청소년,1,3,20260201
2,일반,20260201,23,8호선,2828,821,남위례,2,후불카드,01,일반,18,58,20260201
3,기타,20260201,23,8호선,2828,821,남위례,1,선불카드,99,기타,0,2,20260201
4,노인/약자,20260201,23,8호선,2828,821,남위례,1,선불카드,100,우대권,0,2,20260201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,일반,20260201,23,8호선,2815,814,잠실(송파구청),B,정기권카드,01,일반,0,0,20260201
96,노인/약자,20260201,23,8호선,2815,814,잠실(송파구청),2,후불카드,100,우대권,1,1,20260201
97,학생/청소년,20260201,23,8호선,2815,814,잠실(송파구청),2,후불카드,04,청소년,3,0,20260201
98,일반,20260201,23,8호선,2815,814,잠실(송파구청),2,후불카드,01,일반,141,40,20260201



📊 [검증] 샘플 데이터 그룹별 분포
UserGroup
일반        32
학생/청소년    28
노인/약자     27
기타        13
Name: count, dtype: int64


In [8]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 1. 환경 변수 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# (테스트용 키)
if not API_KEY:
    API_KEY = "6f6e596542617a7533366d6278685a"

# 2. API 호출 설정
# 그룹핑 테스트를 위해 충분한 데이터(1000개)를 가져온 후 100개만 자릅니다.
SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 1000
TARGET_DATE = "20260201" # 테스트 성공했던 날짜 (어제 기준)

url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{TARGET_DATE}"

try:
    response = requests.get(url)
    data = response.json()
    
    # 3. 데이터프레임 변환
    if 'response' in data and 'body' in data['response']:
        items = data['response']['body']['items']['item']
        df = pd.DataFrame(items)
        
        # 4. 데이터 전처리
        # (1) 숫자형 변환 (rideNope, gffNope)
        df['rideNope'] = df['rideNope'].astype('int64')
        df['gffNope'] = df['gffNope'].astype('int64')
        
        # (2) UserGroup 생성 로직 (요청하신 4개 분류)
        def get_user_group(code):
            if code == '01':
                return '일반'
            elif code in ['02', '03', '04']:
                return '어린이/학생/청소년'
            elif code in ['06', '100']:
                return '노인/약자'
            else:
                return '기타'
        
        df['UserGroup'] = df['trnscdUserSeCd'].apply(get_user_group)
        
        # 5. 요청하신 컬럼만 선택 및 정렬
        target_columns = [
            'UserGroup', 
            'pasngDe', 
            'pasngHr', 
            'lineNm', 
            'stnCd', 
            'stnNm', 
            'trnscdUserSeCd', 
            'rideNope', 
            'gffNope'
        ]
        
        # 상위 100개 추출
        result_df = df[target_columns].head(100)
        
        # 6. 출력 설정 (잘림 방지)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)
        
        print(f"✅ 데이터 추출 완료 (총 {len(result_df)}행)")
        display(result_df)
        
    else:
        print("❌ 데이터 구조가 예상과 다릅니다. (response > body > items > item 없음)")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

✅ 데이터 추출 완료 (총 100행)


,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNm,trnscdUserSeCd,rideNope,gffNope
0,노인/약자,20260201,23,8호선,2828,남위례,100,0,0
1,어린이/학생/청소년,20260201,23,8호선,2828,남위례,04,1,3
2,일반,20260201,23,8호선,2828,남위례,01,18,58
3,기타,20260201,23,8호선,2828,남위례,99,0,2
4,노인/약자,20260201,23,8호선,2828,남위례,100,0,2
...,...,...,...,...,...,...,...,...,...
95,일반,20260201,23,8호선,2815,잠실(송파구청),01,0,0
96,노인/약자,20260201,23,8호선,2815,잠실(송파구청),100,1,1
97,어린이/학생/청소년,20260201,23,8호선,2815,잠실(송파구청),04,3,0
98,일반,20260201,23,8호선,2815,잠실(송파구청),01,141,40


In [13]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv

# 1. 환경 변수 로드
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")

# (테스트용 키)
if not API_KEY:
    API_KEY = "6f6e596542617a7533366d6278685a"

# 2. 날짜 및 API 설정
# 2월 2일 기준, 7일 전인 '2026년 1월 26일' 데이터 조회 시도
TARGET_DATE = "20260126" 
SERVICE = "getStnPsgr"
TYPE = "json"
START_INDEX = 1
END_INDEX = 10  # 10개만 요청

url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{TARGET_DATE}"

print(f"🚀 1월 26일 데이터 조회 시작: {url}")

try:
    response = requests.get(url)
    data = response.json()

    # 3. 응답 결과 확인 및 파싱
    if 'response' in data and 'body' in data['response']:
        items = data['response']['body']['items']['item']
        
        if not items:
            print(f"⚠️ {TARGET_DATE} 일자의 데이터가 비어있습니다. (보관 기간 경과 가능성)")
        else:
            df = pd.DataFrame(items)

            # 4. 데이터 전처리 (UserGroup 생성 및 형변환)
            # (1) 숫자형 변환
            df['rideNope'] = df['rideNope'].astype('int64')
            df['gffNope'] = df['gffNope'].astype('int64')

            # (2) UserGroup 분류 로직
            def get_user_group(code):
                if code == '01': return '일반'
                elif code in ['02', '03', '04']: return '어린이/학생/청소년'
                elif code in ['06', '100']: return '노인/약자'
                else: return '기타'
            
            df['UserGroup'] = df['trnscdUserSeCd'].apply(get_user_group)

            # 5. 요청하신 컬럼만 선택 (순서대로)
            target_cols = [
                'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
                'stnCd', 'stnNm', 'trnscdUserSeCd', 
                'rideNope', 'gffNope'
            ]
            
            result_df = df[target_cols].head(10)

            # 6. 결과 출력
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            
            print(f"✅ {TARGET_DATE} 데이터 수집 성공! (총 {len(result_df)}건)")
            display(result_df)
            
    # 에러 응답 처리 (API 서비스 에러 메시지 확인)
    elif 'RESULT' in data:
        print(f"❌ API 에러 응답: {data['RESULT']['CODE']} - {data['RESULT']['MESSAGE']}")
    else:
        print("❌ 알 수 없는 응답 구조입니다.")

except Exception as e:
    print(f"❌ 시스템 오류: {e}")

🚀 1월 26일 데이터 조회 시작: http://openapi.seoul.go.kr:8088/6f6e596542617a7533366d6278685a/json/getStnPsgr/1/10/20260126
✅ 20260126 데이터 수집 성공! (총 10건)


,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNm,trnscdUserSeCd,rideNope,gffNope
0,어린이/학생/청소년,20260126,23,8호선,2828,남위례,04,1,1
1,일반,20260126,23,8호선,2828,남위례,01,30,68
2,기타,20260126,23,8호선,2828,남위례,99,0,3
3,노인/약자,20260126,23,8호선,2828,남위례,100,2,4
4,어린이/학생/청소년,20260126,23,8호선,2828,남위례,04,3,2
5,일반,20260126,23,8호선,2828,남위례,01,20,61
6,기타,20260126,23,8호선,2827,모란,11,1,0
7,노인/약자,20260126,23,8호선,2827,모란,100,0,1
8,어린이/학생/청소년,20260126,23,8호선,2827,모란,04,0,2
9,일반,20260126,23,8호선,2827,모란,01,37,19


In [20]:
import pandas as pd
import numpy as np

# 1. 파일 로드 (인코딩 에러 방지 처리)
file_path = '서울교통공사_역별 일별 시간대별 노인 승하차인원 정보_20241231.csv'

try:
    df_raw = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    df_raw = pd.read_csv(file_path, encoding='cp949')

print("✅ CSV 파일 로드 성공")
print(f"원본 컬럼 목록: {list(df_raw.columns)}")

# 2. 컬럼명 매핑
# '호선' 컬럼이 없으므로 lineNm은 매핑에서 제외하고 나중에 추가함
rename_map = {
    '수송일자': 'pasngDe',
    '역번호': 'stnCd',
    '역명': 'stnNm',
    '승하차구분': 'type' 
}
df_raw.rename(columns=rename_map, inplace=True)

# 3. [Missing Column Handling] 누락된 'lineNm' 컬럼 추가
# DB 스키마에는 lineNm이 필수이므로 '정보없음'으로 임시 채움
if 'lineNm' not in df_raw.columns:
    df_raw['lineNm'] = '정보없음'

# 4. [Melt] Wide to Long 변환
id_vars = ['pasngDe', 'stnCd', 'stnNm', 'type', 'lineNm', '연번'] # 연번은 나중에 버림
# 시간대 컬럼 자동 탐지 ('시간대'라는 단어가 포함된 컬럼만 선택)
value_vars = [col for col in df_raw.columns if '시간대' in col]

df_melted = df_raw.melt(
    id_vars=id_vars, 
    value_vars=value_vars,
    var_name='temp_hr', 
    value_name='count'
)

# 5. [Hour Preprocessing] 시간대 변환 로직 적용
def convert_hour(hr_str):
    if '06시간대이전' in hr_str:
        return 5
    elif '24시간대이후' in hr_str:
        return 0
    else:
        try:
            # '06-07시간대' -> 6
            return int(hr_str.split('-')[0])
        except:
            return -1

df_melted['pasngHr'] = df_melted['temp_hr'].apply(convert_hour)

# 6. [Pivot] 승차/하차 분리
# type 컬럼의 값('승차', '하차')을 컬럼으로 올림
df_pivot = df_melted.pivot_table(
    index=['pasngDe', 'pasngHr', 'lineNm', 'stnCd', 'stnNm'],
    columns='type',
    values='count',
    fill_value=0
).reset_index()

# 컬럼명 정리
df_pivot.rename(columns={'승차': 'rideNope', '하차': 'gffNope'}, inplace=True)
df_pivot.columns.name = None # 인덱스 이름 제거

# 7. 메타데이터 추가 및 포맷팅
df_pivot['UserGroup'] = '노인/약자'
df_pivot['trnscdUserSeCd'] = '06'
df_pivot['pasngDe'] = df_pivot['pasngDe'].astype(str).str.replace('-', '')

# 8. 최종 컬럼 순서 맞추기
target_cols = [
    'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
    'stnCd', 'stnNm', 'trnscdUserSeCd', 
    'rideNope', 'gffNope'
]

result_df = df_pivot[target_cols]

# 9. 결과 확인
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(f"\n✅ 변환 완료! (총 {len(result_df)}행)")
display(result_df.tail(100))

print("\n📋 데이터 타입 및 컬럼 확인")
print(result_df.dtypes)

✅ CSV 파일 로드 성공
원본 컬럼 목록: ['연번', '수송일자', '역번호', '역명', '승하차구분', '06시간대이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시간대이후']

✅ 변환 완료! (총 1993980행)


,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNm,trnscdUserSeCd,rideNope,gffNope
1993880,노인/약자,20241231,23,정보없음,2565,하남시청(덕풍·신장),06,2.0,13.0
1993881,노인/약자,20241231,23,정보없음,2566,하남검단산,06,1.0,15.0
1993882,노인/약자,20241231,23,정보없음,2611,응암,06,9.0,52.0
1993883,노인/약자,20241231,23,정보없음,2612,역촌,06,1.0,3.0
1993884,노인/약자,20241231,23,정보없음,2613,불광,06,4.0,2.0
...,...,...,...,...,...,...,...,...,...
1993975,노인/약자,20241231,23,정보없음,2824,단대오거리,06,3.0,15.0
1993976,노인/약자,20241231,23,정보없음,2825,신흥,06,4.0,8.0
1993977,노인/약자,20241231,23,정보없음,2826,수진,06,1.0,25.0
1993978,노인/약자,20241231,23,정보없음,2827,모란,06,9.0,7.0



📋 데이터 타입 및 컬럼 확인
UserGroup          object
pasngDe            object
pasngHr             int64
lineNm             object
stnCd               int64
stnNm              object
trnscdUserSeCd     object
rideNope          float64
gffNope           float64
dtype: object


In [21]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import time

# 1. 환경 설정
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")
if not API_KEY: API_KEY = "6f6e596542617a7533366d6278685a"

# 2. 어제 날짜 데이터 전수 수집 (검증을 위해 충분한 양 필요)
TARGET_DATE = "20260201" 
SERVICE = "getStnPsgr"
MAX_ROWS = 5000 # 서울시 전체 역 개수 * 시간대 고려 시 충분히 넉넉하게 설정
BATCH_SIZE = 1000

all_items = []
print(f"🚀 [검증 시작] {TARGET_DATE}일자 데이터 스캔 중...")

try:
    for start_idx in range(1, MAX_ROWS, BATCH_SIZE):
        end_idx = start_idx + BATCH_SIZE - 1
        url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE}/{start_idx}/{end_idx}/{TARGET_DATE}"
        
        response = requests.get(url)
        data = response.json()
        
        if 'getStnPsgr' in data and 'row' in data['getStnPsgr']:
            items = data['getStnPsgr']['row']
            all_items.extend(items)
        elif 'response' in data: # 구조가 다를 경우 대비 (표준 공공데이터 포맷)
            items = data['response']['body']['items']['item']
            all_items.extend(items)
        else:
            break # 데이터 끝
            
    df = pd.DataFrame(all_items)
    
    # 3. [핵심 검증 로직] stnCd 하나에 lineNm이 2개 이상인 경우 찾기
    # (1) 역코드, 역명, 호선명만 남기고 중복 제거 (유니크한 역 목록 생성)
    station_master = df[['stnCd', 'stnNm', 'lineNm']].drop_duplicates()
    
    # (2) stnCd별로 호선 개수 세기
    dup_check = station_master.groupby('stnCd')['lineNm'].nunique()
    
    # (3) 호선이 2개 이상인 stnCd 필터링
    problematic_stn_codes = dup_check[dup_check > 1].index.tolist()
    
    print("\n" + "="*50)
    print("🕵️‍♂️ 검증 결과 보고")
    print("="*50)
    
    if len(problematic_stn_codes) > 0:
        print(f"⚠️ 발견됨! 하나의 stnCd가 여러 호선을 가집니다. (총 {len(problematic_stn_codes)}건)")
        
        # 상세 내용 출력
        result = station_master[station_master['stnCd'].isin(problematic_stn_codes)].sort_values('stnCd')
        display(result)
        print("\n>> [결론] PK 설정 시 (stnCd)만으로는 유일성을 보장할 수 없습니다.")
        print(">> (stnCd, lineNm) 또는 (stnCd, stnNm, lineNm) 복합키가 필요합니다.")
        
    else:
        print("✅ 이상 없음. (Clean Data)")
        print("현재 데이터상 모든 stnCd는 오직 하나의 lineNm과 1:1 매핑됩니다.")
        print("즉, 환승역이라도 호선이 다르면 stnCd가 다르게 부여되어 있습니다.")
        
        # 예시로 환승역 하나 보여주기 (서울역)
        print("\n[참고: '서울역'의 호선별 코드 분리 현황]")
        display(station_master[station_master['stnNm'] == '서울역'])

except Exception as e:
    print(f"❌ 오류 발생: {e}")

🚀 [검증 시작] 20260201일자 데이터 스캔 중...

🕵️‍♂️ 검증 결과 보고
✅ 이상 없음. (Clean Data)
현재 데이터상 모든 stnCd는 오직 하나의 lineNm과 1:1 매핑됩니다.
즉, 환승역이라도 호선이 다르면 stnCd가 다르게 부여되어 있습니다.

[참고: '서울역'의 호선별 코드 분리 현황]


,stnCd,stnNm,lineNm
1225,0426,서울역,4호선
2200,0150,서울역,1호선


In [22]:
import requests
import os
from dotenv import load_dotenv
import time

# 1. 환경 설정
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")
if not API_KEY: API_KEY = "6f6e596542617a7533366d6278685a"

SERVICE = "getStnPsgr"
TYPE = "json"
TARGET_DATE = "20260201" # 데이터가 확실히 있는 어제 날짜

def test_request(start, end, description):
    url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{start}/{end}/{TARGET_DATE}"
    count = end - start + 1
    print(f"\n[{description}] {count}건 요청 시도 중... ({start}~{end})")
    
    try:
        start_time = time.time()
        response = requests.get(url, timeout=5)
        duration = time.time() - start_time
        
        data = response.json()
        
        # 성공 여부 체크
        if 'getStnPsgr' in data:
            row_count = len(data['getStnPsgr']['row'])
            print(f"✅ 성공! {row_count}건 수신 완료 (소요시간: {duration:.2f}초)")
            return True
        
        # 에러 체크
        elif 'RESULT' in data:
            code = data['RESULT']['CODE']
            msg = data['RESULT']['MESSAGE']
            print(f"❌ 실패 (API 거부): {code} - {msg}")
            return False
            
    except Exception as e:
        print(f"❌ 시스템 오류: {e}")
        return False

# --- 테스트 실행 ---
print("🚀 API 요청 한계(Capacity) 테스트 시작")

# 1. 표준 제한인 1000건 테스트
test_request(1, 1000, "Case A: 1,000건 (표준)")

# 2. 1000건 초과 시도 (1200건) - 서울시 API는 보통 여기서 막힘
time.sleep(1) # 예의상 1초 대기
test_request(1, 1200, "Case B: 1,200건 (초과 시도)")

print("\n" + "="*50)
print("💡 결론 및 가이드라인")
print("="*50)
print("서울 열린데이터 광장의 공식 가이드는 '1회 1,000건' 입니다.")
print("따라서 반복문(Loop)을 짤 때 step=1000 으로 설정해야 합니다.")

🚀 API 요청 한계(Capacity) 테스트 시작

[Case A: 1,000건 (표준)] 1000건 요청 시도 중... (1~1000)
❌ 시스템 오류: HTTPConnectionPool(host='openapi.seoul.go.kr', port=8088): Read timed out. (read timeout=5)

[Case B: 1,200건 (초과 시도)] 1200건 요청 시도 중... (1~1200)
❌ 시스템 오류: Expecting value: line 1 column 1 (char 0)

💡 결론 및 가이드라인
서울 열린데이터 광장의 공식 가이드는 '1회 1,000건' 입니다.
따라서 반복문(Loop)을 짤 때 step=1000 으로 설정해야 합니다.


In [23]:
import requests
import os
from dotenv import load_dotenv
import time
import json

# 1. 환경 설정
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")
if not API_KEY: API_KEY = "6f6e596542617a7533366d6278685a"

SERVICE = "getStnPsgr"
TYPE = "json"
TARGET_DATE = "20260201" 

def test_request_v2(start, end, description, timeout_sec=30):
    url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{start}/{end}/{TARGET_DATE}"
    count = end - start + 1
    print(f"\n[{description}] {count}건 요청 시도 중... (Timeout={timeout_sec}초)")
    
    try:
        start_time = time.time()
        # 타임아웃을 30초로 설정
        response = requests.get(url, timeout=timeout_sec)
        duration = time.time() - start_time
        
        # HTTP 상태 코드 확인
        if response.status_code != 200:
            print(f"❌ HTTP 에러: {response.status_code}")
            return False

        # JSON 파싱 시도
        try:
            data = response.json()
        except json.JSONDecodeError:
            print(f"❌ JSON 파싱 실패 (서버가 JSON을 주지 않음)")
            print(f"   응답 앞부분: {response.text[:200]}") # 에러 메시지 확인용
            return False
        
        # 데이터 검증
        if 'getStnPsgr' in data:
            row_count = len(data['getStnPsgr']['row'])
            print(f"✅ 성공! {row_count}건 수신 완료 (소요시간: {duration:.2f}초)")
            return True
            
        elif 'response' in data: # 표준 포맷인 경우
            items = data['response']['body']['items']['item']
            print(f"✅ 성공! {len(items)}건 수신 완료 (소요시간: {duration:.2f}초)")
            return True
            
        elif 'RESULT' in data:
            code = data['RESULT']['CODE']
            msg = data['RESULT']['MESSAGE']
            print(f"❌ API 로직 거부: {code} - {msg}")
            return False
            
    except requests.exceptions.Timeout:
        print(f"❌ 타임아웃 발생! ({timeout_sec}초 경과)")
        print("   >> 서버가 응답을 생성하는 데 너무 오래 걸립니다. 요청 단위를 줄여야 합니다.")
        return False
        
    except Exception as e:
        print(f"❌ 기타 시스템 오류: {e}")
        return False

# --- 테스트 실행 ---
print("🚀 API 요청 한계(Capacity) 재테스트 시작")

# 1. 1,000건 (타임아웃 30초)
result_1000 = test_request_v2(1, 1000, "Case A: 1,000건", timeout_sec=30)

# 2. 1,000건이 성공했다면 굳이 1,200건을 테스트하여 서버를 괴롭히지 않습니다.
#    하지만 실패하거나 에러 메시지를 확인하고 싶다면 아래 주석을 해제하세요.
if result_1000:
    print("\n✨ 1,000건 요청이 안정적이라면 이대로 진행하는 것을 권장합니다.")
else:
    print("\n⚠️ 1,000건도 실패한다면 500건으로 줄여야 합니다.")
    time.sleep(2)
    test_request_v2(1, 500, "Case C: 500건 (축소 시도)", timeout_sec=30)

🚀 API 요청 한계(Capacity) 재테스트 시작

[Case A: 1,000건] 1000건 요청 시도 중... (Timeout=30초)
✅ 성공! 1000건 수신 완료 (소요시간: 4.66초)

✨ 1,000건 요청이 안정적이라면 이대로 진행하는 것을 권장합니다.


In [26]:
import requests
import pandas as pd
import pymysql
import os
import time
from datetime import datetime, timedelta
# [수정됨] dbconnect 모듈 임포트
import dbconnect 

# 상수 설정
SERVICE = "getStnPsgr"
TYPE = "json"
BATCH_SIZE = 1000 

# API 키 가져오기 (dbconnect 내부에서 load_dotenv를 하긴 하지만, 여기서도 API_KEY는 필요함)
from dotenv import load_dotenv
load_dotenv()
API_KEY = os.getenv("SEOUL_API_KEY")
if not API_KEY: API_KEY = "6f6e596542617a7533366d6278685a"

# ---------------------------------------------------------
# 🛠️ 함수 정의
# ---------------------------------------------------------

def get_connection():
    """
    dbconnect.py의 MydbConnect 함수를 사용하여 연결 객체 반환
    target_db: 'seoul_urban_lab'
    """
    # dbconnect.py 파일 내부 함수 호출
    conn = dbconnect.MydbConnect(database='seoul_urban_lab')
    
    # 딕셔너리 커서 사용을 위해 설정 변경 (데이터 처리에 편리함)
    # dbconnect는 기본 커서를 반환하므로, 필요시 DictCursor로 동작하게끔 cursor() 호출 때 지정
    return conn

def get_user_group(code):
    """사용자 코드 분류 로직"""
    if code == '01': return '일반'
    elif code in ['02', '03', '04']: return '어린이/학생/청소년'
    elif code in ['06', '100']: return '노인/약자'
    else: return '기타'

def fetch_data(date_str, start_idx, end_idx):
    """API 호출"""
    url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/{TYPE}/{SERVICE}/{start_idx}/{end_idx}/{date_str}"
    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()
        
        if 'getStnPsgr' in data and 'row' in data['getStnPsgr']:
            return data['getStnPsgr']['row']
        elif 'response' in data: 
            return data['response']['body']['items']['item']
        else:
            return []
    except Exception as e:
        print(f"    ❌ [API Error] {e}")
        return []

def save_to_db(conn, df):
    """DB 적재 (Upsert)"""
    if df.empty: return 0
    
    # pymysql.cursors.DictCursor를 명시적으로 사용하지 않아도 Insert에는 문제 없으나,
    # 명확성을 위해 커서 생성
    cursor = conn.cursor(pymysql.cursors.DictCursor)
    
    sql = """
    INSERT INTO subway_traffic_log 
    (UserGroup, pasngDe, pasngHr, lineNm, stnCd, stnNm, trnscdUserSeCd, rideNope, gffNope)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        rideNope = VALUES(rideNope),
        gffNope = VALUES(gffNope)
    """
    
    data_tuples = []
    for _, row in df.iterrows():
        data_tuples.append((
            row['UserGroup'],
            row['pasngDe'],
            int(row['pasngHr']),
            row['lineNm'],
            row['stnCd'],
            row['stnNm'],
            row['trnscdUserSeCd'],
            int(row['rideNope']),
            int(row['gffNope'])
        ))
    
    try:
        cursor.executemany(sql, data_tuples)
        conn.commit()
        return len(data_tuples)
    except Exception as e:
        print(f"    ❌ [DB Error] {e}")
        conn.rollback()
        return 0

# ---------------------------------------------------------
# 🚀 메인 파이프라인 실행
# ---------------------------------------------------------
def run_pipeline():
    # 1. DB 연결 (dbconnect.py 활용)
    conn = get_connection()
    
    # 2. 날짜 루프 설정 (1월 26일 ~ 2월 1일)
    start_date = datetime(2026, 1, 26)
    end_date = datetime(2026, 2, 1)
    
    current_date = start_date
    total_inserted = 0
    
    while current_date <= end_date:
        target_date_str = current_date.strftime("%Y%m%d")
        print(f"\n📅 [Date: {target_date_str}] 데이터 수집 시작...")
        
        start_idx = 1
        daily_total = 0
        
        while True:
            end_idx = start_idx + BATCH_SIZE - 1
            print(f"    - Fetching {start_idx} ~ {end_idx}...", end="\r")
            
            # Extract
            items = fetch_data(target_date_str, start_idx, end_idx)
            
            if not items:
                print(f"    - 데이터 끝 (Page End)")
                break
                
            # Transform
            df = pd.DataFrame(items)
            df['UserGroup'] = df['trnscdUserSeCd'].apply(get_user_group)
            
            # Load
            count = save_to_db(conn, df)
            daily_total += count
            
            if len(items) < BATCH_SIZE:
                break
                
            start_idx += BATCH_SIZE
            time.sleep(0.2)
            
        print(f"    ✨ {target_date_str} 완료! 저장 건수: {daily_total}")
        total_inserted += daily_total
        current_date += timedelta(days=1)
        
    conn.close()
    print("\n" + "="*50)
    print(f"🎉 전체 작업 완료! 총 {total_inserted}건 처리됨.")
    print("="*50)

if __name__ == "__main__":
    run_pipeline()

데이터베이서 연결 시작!
MySQL DB 연결 성공

📅 [Date: 20260126] 데이터 수집 시작...
    ✨ 20260126 완료! 저장 건수: 63090

📅 [Date: 20260127] 데이터 수집 시작...
    ✨ 20260127 완료! 저장 건수: 63589

📅 [Date: 20260128] 데이터 수집 시작...
    ✨ 20260128 완료! 저장 건수: 63636

📅 [Date: 20260129] 데이터 수집 시작...
    ✨ 20260129 완료! 저장 건수: 63927

📅 [Date: 20260130] 데이터 수집 시작...
    ✨ 20260130 완료! 저장 건수: 64828

📅 [Date: 20260131] 데이터 수집 시작...
    ✨ 20260131 완료! 저장 건수: 64489

📅 [Date: 20260201] 데이터 수집 시작...
    ✨ 20260201 완료! 저장 건수: 61567

🎉 전체 작업 완료! 총 445126건 처리됨.


In [27]:
import pandas as pd
import os
import dbconnect # 제공해주신 DB 연결 모듈 사용
import sys

# ---------------------------------------------------------
# 1. 파일 경로 설정 (사용자 환경)
# ---------------------------------------------------------
BASE_DIR = r"C:\Users\Admin\Desktop\hdc2\0202 프로젝트 구축"
CSV_DIR = os.path.join(BASE_DIR, "csv")

# 매핑 파일 경로
MAPPING_FILE = os.path.join(BASE_DIR, "station_master_mapping.csv")

# 처리할 파일 목록 (순서대로 22 -> 23 -> 24)
TARGET_FILES = [
    os.path.join(CSV_DIR, "서울교통공사_역별 일별 시간대별 노인 승하차인원_2022.csv"),
    os.path.join(CSV_DIR, "서울교통공사_일별 역별 시간대별 노인 승하차인원_2023.csv"),
    os.path.join(CSV_DIR, "서울교통공사_역별 일별 시간대별 노인 승하차인원_2024.csv")
]

# ---------------------------------------------------------
# 2. 호선 매핑 데이터 로드 함수
# ---------------------------------------------------------
def load_mapping_table():
    if not os.path.exists(MAPPING_FILE):
        print(f"❌ 매핑 파일이 없습니다: {MAPPING_FILE}")
        sys.exit()
        
    print(f"📂 매핑 테이블 로드 중... ({MAPPING_FILE})")
    df_map = pd.read_csv(MAPPING_FILE, encoding='utf-8')
    
    # Merge를 위해 stnCd를 문자열로 통일
    df_map['stnCd'] = df_map['stnCd'].astype(str).str.strip()
    
    # 필요한 컬럼만 리턴 (역코드, 호선명)
    return df_map[['stnCd', 'lineNm']]

# ---------------------------------------------------------
# 3. 데이터 변환 및 매핑 로직 (핵심)
# ---------------------------------------------------------
def process_file(file_path, df_mapping):
    print(f"\n🔄 파일 처리 시작: {os.path.basename(file_path)}")
    
    # (1) CSV 로드
    try:
        df_raw = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df_raw = pd.read_csv(file_path, encoding='cp949')
        
    # (2) 컬럼명 매핑 (수송일자 -> pasngDe 등)
    rename_map = {
        '수송일자': 'pasngDe',
        '역번호': 'stnCd',
        '역명': 'stnNm',
        '승하차구분': 'type'
    }
    df_raw.rename(columns=rename_map, inplace=True)
    
    # (3) [Mapping] 호선명 채우기
    # 원본 CSV의 stnCd도 문자열로 변환하여 매칭 확률 높임
    df_raw['stnCd'] = df_raw['stnCd'].astype(str).str.strip()
    
    # 기존에 '호선' 컬럼이 있으면 제거 (매핑 데이터로 덮어쓰기 위해)
    if '호선' in df_raw.columns:
        df_raw.drop(columns=['호선'], inplace=True)
    if 'lineNm' in df_raw.columns:
        df_raw.drop(columns=['lineNm'], inplace=True)

    # Left Join 수행
    df_merged = pd.merge(df_raw, df_mapping, on='stnCd', how='left')
    
    # 매핑 안 된 역은 '정보없음' 처리
    df_merged['lineNm'] = df_merged['lineNm'].fillna('정보없음')
    
    # (4) Melt (Wide -> Long)
    # id_vars에 lineNm 포함
    id_vars = ['pasngDe', 'stnCd', 'stnNm', 'type', 'lineNm']
    # '연번'이 있으면 제외
    if '연번' in df_merged.columns:
        # id_vars나 value_vars 어디에도 포함 안 시키면 자동으로 drop 효과
        pass 
        
    # 시간대 컬럼 자동 탐지
    value_vars = [col for col in df_merged.columns if '시간대' in col]
    
    df_melted = df_merged.melt(
        id_vars=id_vars,
        value_vars=value_vars,
        var_name='temp_hr',
        value_name='count'
    )
    
    # (5) 시간대 변환
    def convert_hour(hr_str):
        if '06시간대이전' in hr_str: return 5
        elif '24시간대이후' in hr_str: return 0
        else:
            try:
                return int(hr_str.split('-')[0])
            except:
                return -1

    df_melted['pasngHr'] = df_melted['temp_hr'].apply(convert_hour)
    
    # (6) Pivot (승/하차 분리)
    df_pivot = df_melted.pivot_table(
        index=['pasngDe', 'pasngHr', 'lineNm', 'stnCd', 'stnNm'],
        columns='type',
        values='count',
        fill_value=0
    ).reset_index()
    
    df_pivot.rename(columns={'승차': 'rideNope', '하차': 'gffNope'}, inplace=True)
    df_pivot.columns.name = None
    
    # (7) 메타데이터 추가
    df_pivot['UserGroup'] = '노인/약자'
    df_pivot['trnscdUserSeCd'] = '06'
    # 날짜 포맷 정리 (2022-01-01 -> 20220101)
    df_pivot['pasngDe'] = df_pivot['pasngDe'].astype(str).str.replace('-', '')
    
    # 최종 컬럼 순서
    target_cols = [
        'UserGroup', 'pasngDe', 'pasngHr', 'lineNm', 
        'stnCd', 'stnNm', 'trnscdUserSeCd', 
        'rideNope', 'gffNope'
    ]
    
    return df_pivot[target_cols]

# ---------------------------------------------------------
# 4. DB 적재 함수
# ---------------------------------------------------------
def insert_to_db(conn, df):
    if df.empty: return 0
    
    cursor = conn.cursor()
    
    # 테이블명에 하이픈(-)이 있으므로 반드시 백틱(`) 사용
    sql = """
    INSERT INTO `subway_traffic_log_senior_22-24`
    (UserGroup, pasngDe, pasngHr, lineNm, stnCd, stnNm, trnscdUserSeCd, rideNope, gffNope)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        rideNope = VALUES(rideNope),
        gffNope = VALUES(gffNope)
    """
    
    # DataFrame -> Tuple List 변환
    data_tuples = [tuple(x) for x in df.to_numpy()]
    
    try:
        # 대량 데이터 일괄 입력 (Batch Insert)
        cursor.executemany(sql, data_tuples)
        conn.commit()
        return len(data_tuples)
    except Exception as e:
        print(f"❌ DB 저장 오류: {e}")
        conn.rollback()
        return 0

# ---------------------------------------------------------
# 5. 메인 실행 블록
# ---------------------------------------------------------
def run_batch_etl():
    # 1. 매핑 테이블 준비
    df_mapping = load_mapping_table()
    
    # 2. DB 연결
    print("\n🔌 DB 연결 시도...")
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    total_processed = 0
    
    # 3. 파일별 순차 처리 (2022 -> 2023 -> 2024)
    for file_path in TARGET_FILES:
        if not os.path.exists(file_path):
            print(f"⚠️ 파일 없음 Skip: {file_path}")
            continue
            
        # 변환
        df_final = process_file(file_path, df_mapping)
        print(f"   -> 변환 완료: {len(df_final)}행")
        
        # 적재 (10000개씩 끊어서 넣기 - 메모리 보호)
        CHUNK_SIZE = 10000
        file_inserted_count = 0
        
        print("   -> DB 적재 시작...")
        for i in range(0, len(df_final), CHUNK_SIZE):
            chunk = df_final.iloc[i:i+CHUNK_SIZE]
            cnt = insert_to_db(conn, chunk)
            file_inserted_count += cnt
            print(f"      - {i + cnt} / {len(df_final)} 완료", end="\r")
            
        print(f"\n   ✅ {os.path.basename(file_path)} 적재 완료! (+{file_inserted_count}건)")
        total_processed += file_inserted_count
        
    conn.close()
    print("\n" + "="*50)
    print(f"🎉 모든 작업 완료! 총 {total_processed}건의 데이터가 DB에 저장되었습니다.")
    print("="*50)

if __name__ == "__main__":
    run_batch_etl()

📂 매핑 테이블 로드 중... (C:\Users\Admin\Desktop\hdc2\0202 프로젝트 구축\station_master_mapping.csv)

🔌 DB 연결 시도...
데이터베이서 연결 시작!
MySQL DB 연결 성공

🔄 파일 처리 시작: 서울교통공사_역별 일별 시간대별 노인 승하차인원_2022.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_5040\4034299985.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(file_path, encoding='cp949')


   -> 변환 완료: 1985613행
   -> DB 적재 시작...
      - 1985613 / 1985613 완료
   ✅ 서울교통공사_역별 일별 시간대별 노인 승하차인원_2022.csv 적재 완료! (+1985613건)

🔄 파일 처리 시작: 서울교통공사_일별 역별 시간대별 노인 승하차인원_2023.csv


C:\Users\Admin\AppData\Local\Temp\ipykernel_5040\4034299985.py:49: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(file_path, encoding='cp949')


   -> 변환 완료: 1985601행
   -> DB 적재 시작...
      - 1985601 / 1985601 완료
   ✅ 서울교통공사_일별 역별 시간대별 노인 승하차인원_2023.csv 적재 완료! (+1985601건)

🔄 파일 처리 시작: 서울교통공사_역별 일별 시간대별 노인 승하차인원_2024.csv
   -> 변환 완료: 1993980행
   -> DB 적재 시작...
      - 1993980 / 1993980 완료
   ✅ 서울교통공사_역별 일별 시간대별 노인 승하차인원_2024.csv 적재 완료! (+1993980건)

🎉 모든 작업 완료! 총 5965194건의 데이터가 DB에 저장되었습니다.


In [28]:
import pandas as pd
import dbconnect

def verify_senior_data():
    print("🔍 [Data Verification] 데이터 적재 확인 중...")
    
    # 1. DB 연결
    conn = dbconnect.MydbConnect('seoul_urban_lab')
    
    try:
        # 2. 검증 쿼리 작성
        # 조건: 2022년 2월 23일 / 15시 / 서울역
        sql = """
            SELECT * FROM `subway_traffic_log_senior_22-24`
            WHERE pasngDe = '20220223' 
              AND pasngHr = 15 
              AND stnNm = '서울역'
        """
        
        # 3. 데이터 조회
        df = pd.read_sql(sql, conn)
        
        # 4. 결과 출력
        if df.empty:
            print("❌ 해당 조건의 데이터가 조회되지 않았습니다.")
        else:
            print(f"✅ 조회 성공! (총 {len(df)}건)")
            
            # 보기 좋게 출력 옵션 설정
            pd.set_option('display.max_columns', None)
            pd.set_option('display.width', 1000)
            
            display(df)
            
    except Exception as e:
        print(f"❌ 조회 중 오류 발생: {e}")
        
    finally:
        conn.close()

if __name__ == "__main__":
    verify_senior_data()

🔍 [Data Verification] 데이터 적재 확인 중...
데이터베이서 연결 시작!
MySQL DB 연결 성공
✅ 조회 성공! (총 2건)


C:\Users\Admin\AppData\Local\Temp\ipykernel_5040\3777151086.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conn)


,id,UserGroup,pasngDe,pasngHr,lineNm,stnCd,stnNm,trnscdUserSeCd,rideNope,gffNope,created_at
0,291306,노인/약자,20220223,15,1호선,150,서울역,06,427,311,2026-02-02 12:11:36
1,291416,노인/약자,20220223,15,4호선,426,서울역,06,87,92,2026-02-02 12:11:36
